In [ ]:
from qwen_agent.tools import TOOL_REGISTRY, MCPManager, BaseTool
import json
from ast import literal_eval


def parse_mcp_tools_config(file_path):
    try:
        with open(file_path, 'r') as f:
            content = f.read()
        # 使用 literal_eval 安全地解析 Python 字面量
        data = literal_eval(content)
        return data
    except Exception as e:
        print(f"解析错误: {e}")
        return None

tool = parse_mcp_tools_config("/home/ma-user/work/RL-Factory/envs/configs/data_process.pydata")[0]
# tool = parse_mcp_tools_config("/hpc/home/minruixu/RL-Factory/envs/configs/mcp_tools.pydata")[0]

tools = MCPManager().initConfig(tool)

2025-09-08 22:32:26,045 - mcp_manager.py - 141 - INFO - Initializing MCP tools from mcp servers: ['file_system', 'math', 'posting', 'ticket', 'trading', 'travel', 'vehicle', 'message']
2025-09-08 22:32:26,049 - mcp_manager.py - 370 - INFO - Initializing a MCP stdio_client, if this takes forever, please check the config of this mcp server: file_system


KeyboardInterrupt: 

2025-09-08 22:32:31,237 - mcp_manager.py - 385 - WARNING - Failed in connecting to MCP server: Connection closed


In [1]:
# 读取parquet文件
import pyarrow.parquet as pq

parquet_file = pq.ParquetFile('/home/ma-user/work/RL-Factory/data/BFCL/multi-turn/train.parquet')
data = parquet_file.read().to_pandas()
data.head()


,id,question,golden_answers,data_source,prompt,agent_name,ability,reward_model,extra_info
0,multi_turn_base_0_turn_0_0,Move 'final_report.pdf' within document direct...,"[file_system-cd(folder='document'), file_syste...",BFCL_multi_turn_base,[{'content': 'You are a helpful assistant that...,tool_agent,tool_use,"{'ground_truth': '[""file_system-cd(folder='doc...","{'final_config': '{""file_system"": {""root"": {""w..."
1,multi_turn_base_0_turn_1_0,Perform a detailed search using grep to identi...,"[file_system-cd(folder='temp'), file_system-gr...",BFCL_multi_turn_base,[{'content': 'You are a helpful assistant that...,tool_agent,tool_use,"{'ground_truth': '[""file_system-cd(folder='tem...","{'final_config': '{""file_system"": {""root"": {""w..."
2,multi_turn_base_0_turn_2_0,Upon identifying the requisite 'budget analysi...,[file_system-sort(file_name='final_report.pdf')],BFCL_multi_turn_base,[{'content': 'You are a helpful assistant that...,tool_agent,tool_use,"{'ground_truth': '[""file_system-sort(file_name...","{'final_config': '{""file_system"": {""root"": {""w..."
3,multi_turn_base_0_turn_3_0,Move 'previous_report.pdf' in document directo...,"[file_system-cd(folder='..'), file_system-mv(s...",BFCL_multi_turn_base,[{'content': 'You are a helpful assistant that...,tool_agent,tool_use,"{'ground_truth': '[""file_system-cd(folder='..'...","{'final_config': '{""file_system"": {""root"": {""w..."
4,multi_turn_base_1_turn_0_0,I am alex. Check if the current directory is u...,[file_system-ls(a=True)],BFCL_multi_turn_base,[{'content': 'You are a helpful assistant that...,tool_agent,tool_use,"{'ground_truth': '[""file_system-ls(a=True)""]',...","{'final_config': '{""file_system"": {""root"": {""a..."


In [2]:
extra_info_dict = data['extra_info'].loc(0)
involved_classes = extra_info_dict.get('involved_class', [])
print(involved_classes)

AttributeError: '_LocIndexer' object has no attribute 'get'

In [41]:
# 完整显示 extra_info 的内容
import json

# 遍历每一行的 extra_info 并打印
for idx, row in data.iterrows():
    extra_info = row['extra_info']
    if isinstance(extra_info, str):
        try:
            extra_info = json.loads(extra_info)
            print(f"Row {idx} - extra_info:")
            print(json.dumps(extra_info, indent=4, ensure_ascii=False))
            print("\n" + "="*50 + "\n")
        except json.JSONDecodeError:
            print(f"Row {idx} - extra_info is not valid JSON:")
            print(extra_info)
            print("\n" + "="*50 + "\n")
    else:
        print(f"Row {idx} - extra_info is not a string:")
        print(extra_info)
        print("\n" + "="*50 + "\n")


Row 0 - extra_info is not a string:
{'index': 0, 'initial_config': '{"posting": {"username": "analyst_pro", "password": "Kj8#mP9$vL2", "authenticated": false, "tweets": {"0": {"id": 0, "username": "analyst_pro", "content": "Just finished analyzing the reports!", "tags": ["#analysis", "#reports"], "mentions": []}, "1": {"id": 1, "username": "analyst_pro", "content": "Budget analysis insights coming soon!", "tags": ["#budget", "#analysis", "#insights"], "mentions": []}, "2": {"id": 2, "username": "analyst_pro", "content": "Stay tuned for more updates!", "tags": ["#updates", "#staytuned"], "mentions": []}}, "comments": {}, "retweets": {}, "following_list": ["alice", "bob"], "tweet_counter": 3}, "file_system": {"root": {"workspace": {"type": "directory", "contents": {"document": {"type": "directory", "contents": {"final_report.pdf": {"type": "file", "content": "Year2024 This is the final report content including budget analysis and other sections."}, "previous_report.pdf": {"type": "file",

In [4]:
import json
print("各列的具体数据类型：")
for column in data.columns:
    sample_value = data[column].iloc[0]  # 取每列的第一个值作为样本
    if isinstance(sample_value, str):
        # 检查是否是JSON格式的字符串（可能实际是dict或list）
        try:
            parsed = json.loads(sample_value)
            print(f"{column}: str (实际为 {type(parsed).__name__})")
        except json.JSONDecodeError:
            print(f"{column}: str")
    else:
        print(f"{column}: {type(sample_value).__name__}")

各列的具体数据类型：
id: str
question: str
golden_answers: ndarray
data_source: str
prompt: ndarray
agent_name: str
ability: str
reward_model: dict
extra_info: dict


In [56]:
for col in data['extra_info'][0]:
    print(type(data['extra_info'][0][col]))


data['extra_info'][0]['initial_config']

<class 'int'>
<class 'str'>
<class 'numpy.ndarray'>
<class 'str'>


'{"posting": {"username": "analyst_pro", "password": "Kj8#mP9$vL2", "authenticated": false, "tweets": {"0": {"id": 0, "username": "analyst_pro", "content": "Just finished analyzing the reports!", "tags": ["#analysis", "#reports"], "mentions": []}, "1": {"id": 1, "username": "analyst_pro", "content": "Budget analysis insights coming soon!", "tags": ["#budget", "#analysis", "#insights"], "mentions": []}, "2": {"id": 2, "username": "analyst_pro", "content": "Stay tuned for more updates!", "tags": ["#updates", "#staytuned"], "mentions": []}}, "comments": {}, "retweets": {}, "following_list": ["alice", "bob"], "tweet_counter": 3}, "file_system": {"root": {"workspace": {"type": "directory", "contents": {"document": {"type": "directory", "contents": {"final_report.pdf": {"type": "file", "content": "Year2024 This is the final report content including budget analysis and other sections."}, "previous_report.pdf": {"type": "file", "content": "Year203 This is the previous report content with diffe

In [33]:
initial_config = extra_info_dict.get('initial_config', [])
print(initial_config)

AttributeError: '_LocIndexer' object has no attribute 'get'

In [22]:
print(type(involved_classes))          # 输出 <class 'list'>
print(type(involved_classes[0]))       # 输出 <class 'str'>（假设列表非空）

<class 'numpy.ndarray'>
<class 'str'>
